In [1]:
import os
import dask
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from dask.diagnostics import ProgressBar
from dask.distributed import Client
from geocube.api.core import make_geocube
from tqdm import tqdm

PATH = "D:\World Bank\CLIENT v2"
DATA_RAW = rf"{PATH}\Data\Data_raw"
DATA_PROC = rf"{PATH}\Data\Data_proc"
DATA_OUT = rf"{PATH}\Data\Data_out"
GFD_PATH = rf"D:\Datasets\Global Flood Database\gfd_v1_4"
GPW_PATH = rf"D:\Datasets\Gridded Population of the World"

client = Client()
client

c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 31.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49940,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 31.88 GiB
Comm: tcp://127.0.0.1:49959,Total threads: 1
Dashboard: http://127.0.0.1:49962/status,Memory: 7.97 GiB
Nanny: tcp://127.0.0.1:49943,


In [2]:
import os
from tqdm import tqdm

def load_population_data(bounds=None, generate=False):
    print("Processing Population data...")

    # Select all files in GPW folder
    gpw_path = GPW_PATH
    files = os.listdir(gpw_path)
    files = [f for f in files if f.endswith(".tif")]
    
    # Compile into a single dataset
    dss = []
    for f in tqdm(files):
        
        ds = xr.open_dataset(os.path.join(gpw_path, f), chunks={"x": 43200/2, "y": 17174/2})
        ds["band_data"] = ds["band_data"].astype(np.uint32)
        if bounds is not None:
            ds = ds.sel(
                x=slice(bounds[0], bounds[2]), y=slice(bounds[3], bounds[1])
            )
        if generate:
            with ProgressBar():
                ds.sel(band=1).drop_vars("band").band_data.rio.to_raster(rf"E:\client_v2_data\{f.replace('.tif','_proc.tif')}")
                print(f"Saved {f.replace('.tif','_proc.tif')}")
        
        ds["year"] = int(f.split("_")[5])
        ds = ds.set_coords('year')
        dss += [ds]
        
    population = xr.concat(dss, dim="year")    
    
    # Filter if bounds are provided
    if bounds is not None:
        population = population.sel(
            x=slice(bounds[0], bounds[2]), y=slice(bounds[3], bounds[1])
        )
        
    # Clean band dimension
    population = population.sel(band=1).drop_vars(["band"])
    
    print("Done!")
    return population


In [3]:
import json
import re 
from datetime import datetime

def extract_number_from_filename(filename):
    # Define the regex pattern to extract the number
    match = re.search(r'DFO_(\d+)_', filename)
    if match:
        assert len(match.groups()) == 1
        return int(match.group(1))
    return None

def extract_year_from_filename(filename):
    import re

    year = None
    # Define a regular expression pattern to match the dates
    pattern = r'From_(\d{8})_to_(\d{8})'

    # Use re.search to find the dates in the string
    match = re.search(pattern, filename)

    if match:
        # Extract the dates from the matched groups
        date_from = match.group(1)
        date_to = match.group(2)
        
        # Convert to a more readable format if needed
        # Convert strings to datetime.date objects
        date_from_dt = datetime.strptime(date_from, r'%Y%m%d').date()
        date_to_dt = datetime.strptime(date_to, r'%Y%m%d').date()

        # Calculate the average date
        average_date = date_from_dt + (date_to_dt - date_from_dt) / 2
        average_date = average_date.strftime(r'%Y-%m-%d')
        year = average_date.split("-")[0]
    else:
        print("Dates not found in the string.")

    return year


def check_word_in_json_files(directory, word):
    word = word.upper()
    files_checked = 0
    files_containing_word = []
    
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            files_checked += 1
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                data = json.load(file)
                
                # Check if the word is in any of the values
                for value in data.values():
                    if isinstance(value, str) and word in value.upper():
                        files_containing_word += [filepath]
                        break
                    elif isinstance(value, list) and any(word in str(item).upper() for item in value):
                        files_containing_word += [filepath]
                        break
    
    print(f"Files checked: {files_checked}")
    print(f"Files containing '{word}': {len(files_containing_word)}")
    
    ids = [extract_number_from_filename(f) for f in files_containing_word]
    return ids

def process_gdw_file(ds):
    masked = xr.where(ds.sel(band=5)==1, 0, ds.sel(band=1))
    return masked

def regrid_floods_to_population(floods, like):
    # Calculate the resolution ratios
    like = like.sel(x=slice(floods['x'].min(), floods['x'].max()), y=slice(floods['y'].max(), floods['y'].min()))
    target_lat = like.y
    target_lon = like.x

    floods_xslice = slice(floods['x'].min(), floods['x'].max())
    floods_yslice = slice(floods['y'].max(), floods['y'].min())
    lat_ratio = len(floods['y']) // len(target_lat.sel(y=floods_yslice))
    lon_ratio = len(floods['x']) // len(target_lon.sel(x=floods_xslice))
    
    # Coarsen the floods data by averaging
    coarsened_floods = floods.coarsen(y=int(lat_ratio), x=int(lon_ratio), boundary='trim').mean()
    
    # Rename dimensions to match target grid
    coarsened_floods = coarsened_floods.interp_like(like, method='linear', kwargs={'fill_value': 0})
    return coarsened_floods

## Genero archivos por shock:

Agrega los datos de floods en la misma dimensión que Griddeed Population of The World: 30-arc sec. Reduce la resolución de la grilla de floods al tamaño de la de población, promedidando cada 3 celdas. Entonces, el punto de la grilla de población nos va a decir aproximamandemente que % de la población fue afectada, asumiendo que se distribuyen uniformemente en esa celda (i.e. calcula el % del area afectada). Para ajustar correctamente la grilla, interpolamos con población al final

In [4]:
pop = load_population_data(bounds=None, generate=False)
# pop = xr.open_dataset(rf"{GPW_PATH}\gpw_v4_population_count_rev11_2000_30_sec.tif")

Processing Population data...


  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'ee' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
 25%|██▌       | 1/4 [00:02<00:08,  2.99s/it]c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'ee' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'ee' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
c:\Users\ofici\

Done!


In [ ]:
## Procesamiento 1:
#   Abre todas los eventos de inundación, procesa las bandas y downsamplea, generando un tif por evento con la grilla de población	 
files = os.listdir(GFD_PATH)
files = [f for f in files if f.endswith(".tif")]
population_grid = xr.zeros_like(pop.sel(year=2000))

dss = {}
tasks = []
for i, f in tqdm(enumerate(files), total=len(files)):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        
        year = extract_year_from_filename(f)
        shockid = extract_number_from_filename(f)
        

        ds = xr.open_dataset(os.path.join(GFD_PATH, f), engine="rasterio", chunks={"x": 1000, "y": 1000})
        ds = process_gdw_file(ds)
        # x_bounds = slice(ds.x.min(), ds.x.max())
        # y_bounds = slice(ds.y.max(), ds.y.min())
        # pop_slice = pop.sel(year=2000, x=x_bounds, y=y_bounds)
        ds_aggregated = regrid_floods_to_population(ds, population_grid)
        ds_aggregated = ds_aggregated.assign_coords({"year": year})
        ds_aggregated = ds_aggregated.assign_coords({"id": shockid})
        ds_aggregated = ds_aggregated.to_netcdf(os.path.join(DATA_PROC, "gfd_proc", f"{f.replace('.tif', f'_proc.nc')}"), compute=False)

        if year not in dss.keys():
            dss[year] = [ds_aggregated]
        else:
            dss[year] += [ds_aggregated]

In [ ]:
for year, dss_year in tqdm(dss.items()):
    dask.compute(*dss_year)
# tasks += [xr.concat(dss_year, dim="id").max(dim="id").to_netcdf(os.path.join(DATA_PROC, "gfd_proc", f"{year}_out.nc"), compute=False)]

In [5]:
## Paso 2:
#   Agrega los tif procesados en un solo archivo, calculando la máxima cantidad de personas afectadas cada año

# Genera grilla vacía donde voy relleando los datos de cada evento
grid = xr.zeros_like(pop.sel(year=2000))
grid = grid.assign_coords({"year": "999"})
grid = grid.assign_coords({"id": 999})
grid["band_data"] = grid["band_data"].astype(np.float32)
grid.to_netcdf(os.path.join(DATA_PROC, "empty_grid.nc"))

grid = xr.open_dataset(os.path.join(DATA_PROC, "empty_grid.nc"), chunks={"x": 1000, "y": 1000})

processed_files = os.listdir(os.path.join(DATA_PROC, "gfd_proc"))
processed_files = [f for f in processed_files if f.endswith("_proc.nc")]
dss = {}
# Normalizo la grilla para todos los eventos, de extensión global
for f in tqdm(processed_files):
    year = extract_year_from_filename(f)
    ds_grid = xr.zeros_like(grid)
    ds = xr.open_dataset(os.path.join(DATA_PROC, "gfd_proc", f), chunks={"x": 1000, "y": 1000})
    ds_grid.loc[{"x":slice(ds.x.min(), ds.x.max()), "y":slice(ds.y.max(), ds.y.min())}] = ds.fillna(0) 
    ds_grid["year"] = year
    if year not in dss.keys():
        dss[year] = [ds_grid]
    else:
        dss[year] += [ds_grid]
        
# Para cada año, concateno y calculo la máxima cantidad de personas afectadas
tasks = []
for year, dss_year in tqdm(dss.items()):
    # Calculo el máximo (i.e., si hubo flood) en cada celda
    xr.concat(dss_year, dim="id").max(dim="id").to_netcdf(os.path.join(DATA_PROC, "gfd_proc", f"{year}_out.nc"))
    print(f'Saved {os.path.join(DATA_PROC, "gfd_proc", f"{year}_out.nc")}')

  5%|▌         | 1/19 [01:11<21:29, 71.63s/it]

Saved D:\World Bank\CLIENT v2\Data\Data_proc\gfd_proc\2000_out.nc


 11%|█         | 2/19 [03:23<30:23, 107.24s/it]

Saved D:\World Bank\CLIENT v2\Data\Data_proc\gfd_proc\2001_out.nc


 16%|█▌        | 3/19 [08:33<53:16, 199.78s/it]

...

Saved D:\World Bank\CLIENT v2\Data\Data_proc\gfd_proc\2016_out.nc


 95%|█████████▍| 18/19 [1:21:38<03:32, 212.06s/it]

Saved D:\World Bank\CLIENT v2\Data\Data_proc\gfd_proc\2017_out.nc


100%|██████████| 19/19 [1:25:03<00:00, 268.59s/it]

Saved D:\World Bank\CLIENT v2\Data\Data_proc\gfd_proc\2018_out.nc


In [7]:
## Paso 3: compila todo en una unica base
# Para cada año, concateno y calculo la máxima cantidad de personas afectadas
all_ds = []
for year, dss_year in tqdm(dss.items()):
    ds = xr.open_dataset(os.path.join(DATA_PROC, "gfd_proc", f"{year}_out.nc"), chunks={"x":1000, "y":1000})
    ds["year"] = year
    all_ds += [ds]
full_ds = xr.concat(all_ds, dim="year")
full_ds.to_netcdf(os.path.join(DATA_OUT, "floods_by_year.nc"))

  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 111.25 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
# Ver progreso en el dashboard: http://127.0.0.1:8787/status
dask.compute(*tasks)